In [ ]:
import pandas as pd

# Load the CSV
file_path = "alloys_with_description.csv"
df = pd.read_csv(file_path)

# Select relevant columns
data = df[['Alloy', 'alloy_description']].dropna()

# Create a mapping: Alloy Name -> Description
data_dict = dict(zip(data['Alloy'], data['alloy_description']))


FileNotFoundError: [Errno 2] No such file or directory: '/content/alloys_with_description.csv'

In [ ]:
data_dict = dict(zip(data['Alloy'], data['alloy_description'])) # This creates a dictionary
# To display the first 5 entries in the dictionary
for i, (key, value) in enumerate(data_dict.items()):
    if i < 5:
        print(f"{key}: {value}")
    else:
        break

Al0.5NbTaTiV: The alloy Al0.5NbTaTiV contains 5 elemental compositions: Al (0.111111111), Ti (0.222222222), V (0.222222222), Nb (0.222222222), Ta (0.222222222) and without heat treatment,showing a BCC microstructure and BCC_SS phases.
Al0.75MoNbTiV: The alloy Al0.75MoNbTiV contains 5 elemental compositions: Al (0.157894737), Ti (0.210526316), V (0.210526316), Nb (0.210526316), Mo (0.210526316) and without heat treatment,showing a BCC microstructure and BCC_SS phases.
Al0.25MoNbTiV: The alloy Al0.25MoNbTiV contains 5 elemental compositions: Al (0.058823529), Ti (0.235294118), V (0.235294118), Nb (0.235294118), Mo (0.235294118) and without heat treatment,showing a BCC microstructure and BCC_SS phases.
Al0.25NbTaTiV: The alloy Al0.25NbTaTiV contains 5 elemental compositions: Al (0.058823529), Ti (0.235294118), V (0.235294118), Nb (0.235294118), Ta (0.235294118) and without heat treatment,showing a BCC microstructure and BCC_SS phases.
Al0.2MoTaTiV: The alloy Al0.2MoTaTiV contains 5 elemen

In [ ]:
alloy_names = list(data_dict.keys())
alloy_descriptions = list(data_dict.values())

In [ ]:
pip install pandas sentence-transformers faiss-cpu transformers


#Generate embeddings using all-MiniLM-L6-v2

In [ ]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
alloy_descriptions = list(data_dict.values())
embeddings = embedding_model.encode(alloy_descriptions, convert_to_tensor=True)


#Generate embeddings using E5

In [ ]:
from sentence_transformers import SentenceTransformer


e5_model = SentenceTransformer('intfloat/e5-large-v2')

e5_inputs = [f"passage: {desc}" for desc in alloy_descriptions]
e5_embeddings = e5_model.encode(e5_inputs, convert_to_tensor=True)


#SciBERT Embeddings

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch


scibert_tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
scibert_model = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased')


def get_scibert_embeddings(texts):
    embeddings = []
    for text in texts:
        tokens = scibert_tokenizer(text, padding=True, truncation=True, return_tensors="pt")
        with torch.no_grad():
            output = scibert_model(**tokens)
            embeddings.append(output.last_hidden_state.mean(dim=1).squeeze(0).numpy())
    return embeddings

scibert_embeddings = torch.tensor(get_scibert_embeddings(alloy_descriptions))


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/228k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/442M [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
<ipython-input-13-02d0af0998eb>:18: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  scibert_embeddings = torch.tensor(get_scibert_embeddings(alloy_descriptions))


#Store Embedding in FAISS

In [ ]:
import faiss
import numpy as np

# E5 index
e5_dimension = e5_embeddings.shape[1]
e5_index = faiss.IndexFlatL2(e5_dimension)
e5_index.add(np.array(e5_embeddings))

# # SciBERT index
# scibert_dimension = scibert_embeddings.shape[1]
# scibert_index = faiss.IndexFlatL2(scibert_dimension)
# scibert_index.add(np.array(scibert_embeddings))

# # MiniLM index
# minilm_dimension = embeddings.shape[1]
# minilm_index = faiss.IndexFlatL2(minilm_dimension)
# minilm_index.add(embeddings.cpu().numpy())

In [ ]:
import faiss

# Specify the file path to save the index
e5_index_file = "e5_faiss_index.bin"

# Save the FAISS index
faiss.write_index(e5_index, e5_index_file)

print(f"FAISS index saved to {e5_index_file}")


FAISS index saved to e5_faiss_index.bin


In [ ]:
import pickle

# Save the metadata (alloy names and descriptions)
metadata_file = "metadata.pkl"
with open(metadata_file, "wb") as f:
    pickle.dump({"alloy_names": alloy_names, "alloy_descriptions": alloy_descriptions}, f)

print(f"Metadata saved to {metadata_file}")


Metadata saved to metadata.pkl


In [ ]:
from google.colab import files

# Download the indices
files.download(e5_index_file)
# Download the metadata
files.download(metadata_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install sentence-transformers faiss-cpu transformers torch accelerate

import os
import pickle
import faiss
import torch
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline

# Configuration (Adjust these paths)
MODEL_NAME = "google/flan-t5-small"  # Smaller, more compatible model
EMBEDDING_MODEL = 'intfloat/e5-small-v2' # Use the same model for consistency
INDEX_PATH = "/content/Metallurgy FAISS Index.bin"
METADATA_PATH = "/content/Metallurgy Metadata.pkl"
DEVICE = "cpu"  # No GPU support for bitsandbytes, so use CPU

print(f"Using device: {DEVICE}")



def load_data(metadata_path, index_path):
    """Loads embeddings and metadata."""
    with open(metadata_path, "rb") as f:
        metadata = pickle.load(f)

    index = faiss.read_index(index_path)
    return metadata, index


def get_relevant_chunks(query, index, metadata, top_k=3):
    """Retrieves relevant text chunks using FAISS."""
    # Use the same embedding model that was used to create the index (e.g., E5)
    # embedding_model = SentenceTransformer(EMBEDDING_MODEL).to(DEVICE)
    embedding_model = SentenceTransformer('intfloat/e5-large-v2').to(DEVICE)  # Assuming E5 was used to create the index
    query_embedding = embedding_model.encode(query)

    # Convert the query embedding to the correct data type if needed
    query_embedding = query_embedding.astype('float32')  # faiss often expects float32

    # Ensure that the shape of the query embedding is correct
    query_embedding = query_embedding.reshape(1, -1)  # Reshape to a 2D array

    # Before searching, check the dimensions:
    print(f"Index dimension: {index.d}, Query embedding dimension: {query_embedding.shape[1]}")

    D, I = index.search(query_embedding, top_k)
    retrieved_chunks = [metadata["alloy_descriptions"][i] for i in I[0]] # Get descriptions from metadata
    return retrieved_chunks


def generate_response(query, context, model_name=MODEL_NAME):
    """Generates a response using the language model with RAG."""
    try:
        qa_pipeline = pipeline(
            "text2text-generation",
            model=model_name,
            device=DEVICE
        )
    except Exception as e:
        print(f"Error loading model: {e}")
        return "Error loading the model. Try a smaller model or restart the runtime."

    prompt = f"""Use the following context to answer the question at the end. If the context doesn't contain the answer, say that you don't know.

    Context:
    {' '.join(context)}

    Question: {query}
    Answer:"""

    try:
        result = qa_pipeline(prompt, max_length=512)
        return result[0]["generated_text"].strip()
    except Exception as e:
        print(f"Error during generation: {e}")
        return "An error occurred during response generation."

if __name__ == "__main__":
    try:
        metadata, index = load_data(METADATA_PATH, INDEX_PATH)
    except FileNotFoundError:
        print(f"Error: index or metadata files not found at {DATA_DIR}. Please ensure the files are uploaded to Google Drive in the correct folder.")
        exit()  # Stop execution if files aren't found.

    while True:
        query = input("Enter your query (or type 'exit'): ")
        if query.lower() == "exit":
            break

        relevant_chunks = get_relevant_chunks(query, index, metadata)
        response = generate_response(query, relevant_chunks)
        print("Response:", response)

Using device: cpu
Enter your query (or type 'exit'): What are the compositions of the alloy: Al0.5NbTaTiV?
Index dimension: 1024, Query embedding dimension: 1024


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

...

Response: BCC microstructure and BCC_SS phases
Enter your query (or type 'exit'): What is the heat treatment for the alloy Hf0.75NbTa0.5Ti1.5Zr1.25
Index dimension: 1024, Query embedding dimension: 1024


Device set to use cpu


Response: homogenization process
Enter your query (or type 'exit'): Name on the elements that composes the alloy Hf0.75NbTa0.5Ti1.5Zr1.25
Index dimension: 1024, Query embedding dimension: 1024


Device set to use cpu


Response: (4).
Enter your query (or type 'exit'): exit
